In [13]:
#import
!pip install ultralytics==8.0.20
import ultralytics
from ultralytics import YOLO

In [3]:
#to use drive files
from google.colab import drive
drive.mount("/content/gdrive/")
%cd /content/gdrive/MyDrive/yolo/yolov8

Mounted at /content/gdrive/
/content/gdrive/MyDrive/yolo/yolov8


In [4]:
#train YOLO on custom data
!yolo task=detect mode=train model=yolov8m.pt data=custom_data.yaml epochs=25 imgsz=800 plots=True

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=custom_data.yaml, epochs=25, patience=50, batch=16, imgsz=800, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, wo

In [5]:
#evaluate YOLO custom model
!yolo task=detect mode=val model=runs/detect/train11/weights/best.pt data=custom_data.yaml

2024-01-11 13:09:01.330802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 13:09:01.330862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 13:09:01.332246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-11 13:09:02.467256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/gdrive/MyDrive/yolo/yolov8/data/val/labels.cache..

In [6]:
#predict detection
#annoted images can be viewed in the folder "predict"
!yolo task=detect mode=predict model=runs/detect/train11/weights/best.pt conf=0.6 source=ID7726_Carabidae save=True imgsz=1920

2024-01-11 13:10:41.492017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 13:10:41.492072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 13:10:41.493408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-11 13:10:42.610906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
image 1/72 /content/gdrive/MyDrive/yolo/yolov8/ID7726_Carabidae/entomo_X0

In [8]:
#load the custom model
model = YOLO('runs/detect/train11/weights/best.pt')
#get bounding box data
data_test = 'ID7726_Carabidae'
results = model.predict(source=data_test, conf=0.6, imgsz=1920)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


In [11]:
###GEMBLOUX METHOD###

import os

#coordinates of the counting zone
y_i = 255
y_f = 825
x_i = 500
x_f = 1380

#determine the height of the box (number of images in y) according to the total number of images in the folder
if len(os.listdir(data_test))>36:
    if len(os.listdir(data_test))>42:
        height_box = 12
    else:
        height_box = 7
else:
    height_box = 6

#width of the box (number of images in x) is always 6
width_box = 6

#part and insect counter
count_part = 0
count_nb = 0

#for every part of the box
for i in range(len(os.listdir(data_test))):
  #x and y coordinates of the part
  x_part = (count_part//height_box)
  y_part = (count_part%height_box)

  #create local variables to modify the zone if needed
  local_y_i = y_i
  local_y_f = y_f
  local_x_i = x_i
  local_x_f = x_f

  #if it is an edge, extend the zone up to it
  if(x_part == 0):
      local_x_i = 0
  if(x_part == width_box-1):
      local_x_f = 1920
  if(y_part == 0):
      local_y_i = 0
  if(y_part == height_box-1):
      local_y_f = 1080

  #for every predictio=n
  for j in range(len(results[i].boxes.xyxy)):

    #take x and y coordinates of upper left corner the bounding box
    x_i_det = int(results[i][j].boxes.xyxy[0][0].item())
    y_i_det = int(results[i][j].boxes.xyxy[0][1].item())

    #taken into account if is situated in the counting zone
    if x_i_det > local_x_i and x_i_det < local_x_f and y_i_det > local_y_i and y_i_det < local_y_f:
      count_nb += 1

  count_part +=1
print('Number of insects : '+str(count_nb))

Number of insects : 184


In [12]:
###AFRICAMUSEUM METHOD###

#array to store the number of insects in each image of the folder
tab = []

#for every image
for i in range (len(results)):
  #store the number of predictions in the array
  tab.append(len(results[i]))

print(tab)

[5, 8, 7, 8, 8, 8, 8, 8, 7, 10, 6, 2, 10, 14, 13, 13, 15, 15, 14, 15, 14, 14, 8, 3, 8, 13, 15, 12, 13, 10, 6, 9, 12, 14, 9, 3, 8, 14, 12, 9, 13, 12, 7, 10, 11, 11, 11, 3, 7, 11, 8, 9, 12, 8, 10, 14, 12, 12, 10, 3, 4, 5, 5, 4, 3, 7, 7, 8, 7, 6, 7, 3]
